In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import scipy.stats as stats

In [ ]:
#df = pd.read_csv('/root/bryton/github/cv_research/bryton/ml_sealice/2016.csv')
df = pd.read_csv('/root/bryton/github/cv_research/bryton/ml_sealice/combined_2016_2018.csv')


In [ ]:
df
delicings = np.where(df['mechanicalRemoval'].isnull() == False)[0]

num_delicings = len(delicings)
num_site_weeks = len(df)

num_delicings / num_site_weeks

# Average time between delicings
# Number of delicings
# Pull specifically for Erko

In [ ]:
num = df.loc[df['avgAdultFemaleLice'] < 0.5, 'avgAdultFemaleLice']
#num = df.loc[df['avgMobileLice'] < 1.5, 'avgMobileLice']
#num = df.loc[df['avgStationaryLice'] < 0.5, 'avgStationaryLice']

dist = dist=stats.distributions.beta
res = num
#res = np.log(1 + num)

fig = sm.qqplot(res, dist=dist, fit=True, line='45')

plt.show()

In [ ]:
dist= dist=stats.distributions.expon

In [ ]:
print(df.iloc[0])

plt.hist(df.loc[df['avgAdultFemaleLice'] < 1.5, 'avgAdultFemaleLice'], bins='auto', density=True)
plt.show()

plt.hist(df.loc[df['avgMobileLice'] < 5, 'avgMobileLice'], bins='auto', density=True)
plt.show()

plt.hist(df.loc[df['avgStationaryLice'] < 3, 'avgStationaryLice'], bins='auto', density=True)
plt.show()

print(np.mean(df['avgAdultFemaleLice']))
print(np.median(df['avgAdultFemaleLice']))

print(np.mean(df['avgMobileLice']))
print(np.median(df['avgMobileLice']))

print(np.mean(df['avgStationaryLice']))
print(np.median(df['avgStationaryLice']))

for i in range(0, 100, 1):
    p1 = np.percentile(df['avgAdultFemaleLice'], i) # return 50th percentile, e.g median.
    p2 = np.percentile(df['avgMobileLice'], i) # return 50th percentile, e.g median.
    p3 = np.percentile(df['avgStationaryLice'], i) # return 50th percentile, e.g median.
    print('%0.2f: %0.2f, %0.2f, %0.2f' % (i, p1, p2, p3))

In [ ]:
#my_df = df.loc[:,'avgAdultFemaleLice':'seaTemperature']
my_df = df.loc[:,'hasReportedLice':'seaTemperature']

new_df = my_df.copy()
norm_df = my_df.copy()

for x in my_df.columns:
    my_mean = my_df[x].median()
    my_std = my_df[x].std()
    my_iqr = stats.iqr(my_df[x])
    
    my_row = my_df[x].copy()
    new_df[x] = my_row / my_iqr
    norm_df[x] = (my_row - my_mean) / my_std

In [ ]:
#good_indices_1 = (my_df == 0).any(axis=1) == False
good_indices_2 = my_df['avgAdultFemaleLice'] < 1.5
good_indices_3 = my_df['avgMobileLice'] < 5
good_indices_4 = my_df['avgStationaryLice'] < 3

good_indices = good_indices_2 & good_indices_3 & good_indices_4

print(np.sum(good_indices) / len(good_indices))

my_df_X = new_df.filter(['avgAdultFemaleLice', 'avgMobileLice', 'seaTemperature', 'hasMechanicalRemoval', 'hasBathTreatment', 'hasInFeedTreatment', 'hasCleanerFishDeployed'], axis=1)
my_df_Y = new_df['avgStationaryLice'][good_indices]

my_df_X = my_df_X.loc[good_indices, :]

my_df_X = np.log(1 + my_df_X)
my_df_Y = my_df_Y
#my_df_Y = np.log(1 + my_df_Y)

my_df_X = np.hstack((my_df_X, my_df_X ** 2, np.log(1 + my_df_X)))

myX = sm.add_constant(my_df_X)
myY = np.log(1 + my_df_Y)

model = sm.OLS(myY, myX).fit()

res = model.resid
fig = sm.qqplot(res, dist=stats.distributions.norm, fit=True, line='45')

plt.show()

print(model.summary())

In [ ]:
plt.scatter(my_df_X[:,0], my_df_Y)
plt.show()

In [ ]:
predY = model.predict(myX)
error = predY - myY

avg_error = np.median(np.abs(error))

print(avg_error)

plt.scatter(myY, predY)
plt.xlabel('Actual stationary')
plt.ylabel('Predicted stationary')
plt.show()

In [ ]:
res_X = predY
res_Y = predY - myY

res_myX = sm.add_constant(res_X)
res_myY = res_Y

res_model = sm.OLS(res_myY, res_myX).fit_regularized(L1_wt=0)
res_predY = res_model.predict(res_myX)

print(res_model.summary())

plt.scatter(res_X, res_Y)
plt.plot(res_X, res_predY, color='red')
plt.show()

In [ ]:
plt.scatter(np.exp(myY), np.exp(predY))
plt.xlabel('Actual stationary')
plt.ylabel('Predicted stationary')
plt.show()

error = np.exp(predY) - np.exp(myY)

avg_error = np.median(np.abs(error))

print(avg_error)

In [ ]:
# Trying a different model that is time-series based

In [ ]:
subset = my_df.loc[df['localityNo'] == 11756]

print(subset)

plt.plot(subset['avgAdultFemaleLice'])
plt.show()

In [ ]:
print(subset.shape)

subset_v = subset.values
print(subset_v.shape)

diff = subset_v[1:21] - subset_v[0:20]

diff_myX = np.hstack((subset_v[1:21,0:1], subset_v[0:20,:]))
diff_myY = subset_v[1:21, 2]

diff_model = sm.OLS(diff_myY, diff_myX).fit()
diff_predY = diff_model.predict(diff_myX)

print(diff_model.summary())

plt.scatter(diff_myY, diff_predY)
plt.show()

In [ ]:
localityNumbers = np.unique(df['localityNo'])

In [ ]:
# Original model

'''
all_diff_myX = None
all_diff_myY = None

for localityNo in localityNumbers:
    subset = my_df.loc[df['localityNo'] == localityNo]
    subset_v = subset.values
    num_obs = subset_v.shape[0]

    diff = subset_v[1:num_obs] - subset_v[0:(num_obs - 1)]
    
    diff_myX = np.hstack((subset_v[1:num_obs,0:2], subset_v[0:(num_obs - 1),0:3], (subset_v[1:num_obs,0:2] - subset_v[0:(num_obs - 1),0:2]) / subset_v[0:(num_obs - 1),0:2]))
    #diff_myY = subset_v[1:num_obs, 2]
    diff_myY = (subset_v[1:num_obs, 2] - subset_v[0:(num_obs - 1),2]) #/ subset_v[0:(num_obs - 1),2]
    
    if all_diff_myX is None:
        all_diff_myX = diff_myX
        all_diff_myY = diff_myY
    else:
        all_diff_myX = np.vstack((all_diff_myX, diff_myX))
        all_diff_myY = np.concatenate((all_diff_myY, diff_myY))
'''

In [ ]:
subset_v = my_df.loc[df['localityNo'] == localityNumbers[0]]
subset_v.iloc[1]

In [ ]:
subset_v.iloc[0]
subset_v.values[:,0:6]
#subset_v.values[:,6:10]

In [ ]:
# New model

all_diff_myX = None
all_diff_myY = None

for localityNo in localityNumbers:
    subset = my_df.loc[df['localityNo'] == localityNo]
    subset_r = subset.values[:,0:6]
    subset_v = subset.values[:,6:10]
    num_obs = subset_v.shape[0]
    
    if num_obs < 3:
        print('Not enough samples')
        continue
        
    diff_myX = []
    diff_myY = []
    
    for i in range(num_obs - 2):
        X = np.concatenate((subset_v[i+1,0:2], subset_v[i+1,0:2] * subset_v[i+1,3:4], subset_v[i+2,0:3], subset_v[i+2,0:3] * subset_v[i+2,3:4], subset_v[i,0:3], subset_v[i,0:3] * subset_v[i,3:4], subset_v[i,3:4]))
        Y = subset_v[i + 1, 2]
        
        if ( subset_r[i, 4] or  subset_r[i + 1, 4] or  subset_r[i + 2, 4]
            or subset_r[i, 1] or subset_r[i + 1, 1] or subset_r[i + 2, 1]
            or subset_r[i, 2] or subset_r[i + 1, 2] or subset_r[i + 2, 2]
            or subset_r[i, 3] or subset_r[i + 1, 3] or subset_r[i + 2, 3]):
            continue
        
        diff_myX.append(X)
        diff_myY.append(Y)
        
    if len(diff_myX) == 0:
        continue
        
    '''
    #diff = subset_v[1:num_obs] - subset_v[0:(num_obs - 1)]
    
    #diff_myX = np.hstack((subset_v[1:num_obs,0:4], subset_v[0:(num_obs - 1),0:2], subset_v[0:(num_obs - 1),3:4]))
    #diff_myX = np.hstack((subset_v[0:(num_obs - 1),0:4], subset_v[1:(num_obs),0:2], subset_v[1:(num_obs),3:4]))
    #diff_myX = np.hstack((subset_v[2:(num_obs),0:2], subset_v[2:(num_obs),3:4], subset_v[1:(num_obs - 1),0:4], subset_v[0:(num_obs - 2),0:4]))
    #diff_myX = np.hstack((subset_v[1:(num_obs - 1),0:2], subset_v[1:(num_obs - 1),3:4], subset_v[2:(num_obs - 0),0:4], subset_v[0:(num_obs - 2),0:4]))
    diff_myX = np.hstack((subset_v[1:(num_obs - 1),0:2], subset_v[1:(num_obs - 1),0:2] * subset_v[1:(num_obs - 1),3:4], subset_v[2:(num_obs - 0),0:3], subset_v[2:(num_obs - 0),0:3] * subset_v[2:(num_obs - 0),3:4], subset_v[0:(num_obs - 2),0:3], subset_v[0:(num_obs - 2),0:3] * subset_v[0:(num_obs - 2),3:4]))
    #diff_myX = np.hstack((subset_v[2:(num_obs - 1),0:2], subset_v[2:(num_obs - 1),3:4], subset_v[3:(num_obs - 0),0:4], subset_v[1:(num_obs - 2),0:4], subset_v[0:(num_obs - 3),0:4]))
    #diff_myY = subset_v[0:(num_obs - 1), 2]
    diff_myY = subset_v[1:(num_obs - 1), 2]
    #diff_myY = subset_v[2:(num_obs - 1), 2]
    #diff_myY = subset_v[2:(num_obs), 2]
    #diff_myY = (subset_v[1:num_obs, 2] - subset_v[0:(num_obs - 1),2]) #/ subset_v[0:(num_obs - 1),2]
    '''
    
    if all_diff_myX is None:
        all_diff_myX = diff_myX
        all_diff_myY = diff_myY
    else:
        all_diff_myX = np.vstack((all_diff_myX, diff_myX))
        all_diff_myY = np.concatenate((all_diff_myY, diff_myY))
        
        
print('done')

print(all_diff_myX.shape)
print(all_diff_myY.shape)

all_diff_myX = np.array(all_diff_myX, dtype=np.float64)

In [ ]:
print('done')

#all_diff_myY[0]

In [ ]:
# Old
# AF: 0, 3, 7 < 1.5
# M: 1, 3, 8 < 5
# F: 5, 9 < 1.5

# New
# AF: 0, 2, 4, 7, 10, 13 < 1.5
# M: 1, 3, 5, 8, 11, 14 < 5
# F: 6, 9, 12, 15 < 1.5

AF_thresh = 1.5
M_thresh = 5
F_thresh = 1.5

# Adult female
gi_AF1 = all_diff_myX[:,0] < AF_thresh
#gi_AF2 = all_diff_myX[:,2] < AF_thresh
gi_AF2 = all_diff_myX[:,4] < AF_thresh
#gi_AF4 = all_diff_myX[:,7] < AF_thresh
gi_AF3 = all_diff_myX[:,10] < AF_thresh
#gi_AF6 = all_diff_myX[:,13] < AF_thresh
gi_AF4 = all_diff_myX[:,0] > 0.00001
#gi_AF8 = all_diff_myX[:,2] > 0.00001
gi_AF5 = all_diff_myX[:,4] > 0.00001
#gi_AF10 = all_diff_myX[:,7] > 0.00001
gi_AF6 = all_diff_myX[:,10] > 0.00001
#gi_AF12 = all_diff_myX[:,13] > 0.00001

# Moving
gi_M1 = all_diff_myX[:,1] < M_thresh
#gi_M2 = all_diff_myX[:,3] < M_thresh
gi_M2 = all_diff_myX[:,5] < M_thresh
#gi_M4 = all_diff_myX[:,8] < M_thresh
gi_M3 = all_diff_myX[:,11] < M_thresh
#gi_M6 = all_diff_myX[:,14] < M_thresh
gi_M4 = all_diff_myX[:,1] > 0.00001
#gi_M8 = all_diff_myX[:,3] > 0.00001
gi_M5 = all_diff_myX[:,5] > 0.00001
#gi_M10 = all_diff_myX[:,8] > 0.00001
gi_M6 = all_diff_myX[:,11] > 0.00001
#gi_M12 = all_diff_myX[:,14] > 0.00001

# Fixed
gi_F1 = all_diff_myX[:,6] < F_thresh
#gi_F2 = all_diff_myX[:,9] < F_thresh
gi_F2 = all_diff_myX[:,12] < F_thresh
#gi_F4 = all_diff_myX[:,15] < F_thresh
gi_F3 = all_diff_myY < F_thresh
gi_F4 = all_diff_myX[:,6] > 0.00001
#gi_F7 = all_diff_myX[:,9] > 0.00001
gi_F5 = all_diff_myX[:,12] > 0.00001
#gi_F9 = all_diff_myX[:,15] > 0.00001
gi_F6 = all_diff_myY > 0.00001


good_indices = gi_AF1 & gi_AF2 & gi_AF3 & gi_AF4 & gi_AF5 & gi_AF6
good_indices = good_indices & gi_M1 & gi_M2 & gi_M3 & gi_M4 & gi_M5 & gi_M6
good_indices = good_indices & gi_F1 & gi_F2 & gi_F3 & gi_F4 & gi_F5 & gi_F6

# Temperature
#gt_temp = all_diff_myX[:,3] < 15
#good_indices = good_indices & gt_temp
'''
# Adult female
gi_AF1 = all_diff_myX[:,0] < AF_thresh
gi_AF2 = all_diff_myX[:,3] < AF_thresh
gi_AF3 = all_diff_myX[:,7] < AF_thresh
gi_AF4 = all_diff_myX[:,0] > 0.00001
gi_AF5 = all_diff_myX[:,3] > 0.00001
gi_AF6 = all_diff_myX[:,7] > 0.00001

# Moving
gi_M1 = all_diff_myX[:,1] < M_thresh
gi_M2 = all_diff_myX[:,3] < M_thresh
gi_M3 = all_diff_myX[:,8] < M_thresh
gi_M4 = all_diff_myX[:,1] > 0.00001
gi_M5 = all_diff_myX[:,3] > 0.00001
gi_M6 = all_diff_myX[:,8] > 0.00001

# Fixed
gi_F1 = all_diff_myX[:,5] < F_thresh
gi_F2 = all_diff_myX[:,9] < F_thresh
gi_F3 = all_diff_myY < F_thresh
gi_F4 = all_diff_myX[:,5] > 0.00001
gi_F5 = all_diff_myX[:,9] > 0.00001
gi_F6 = all_diff_myY > 0.00001

good_indices = gi_AF1 & gi_AF2 & gi_AF3 & gi_AF4 & gi_AF5 & gi_AF6
good_indices = good_indices & gi_M1 & gi_M2 & gi_M3 & gi_M4 & gi_M5 & gi_M6
good_indices = good_indices & gi_F1 & gi_F2 & gi_F3 & gi_F4 & gi_F5 & gi_F6
'''

print(np.sum(good_indices) / len(good_indices))

all_filtered_myX = all_diff_myX[good_indices, :]
all_filtered_myY = all_diff_myY[good_indices]

print(np.sum(np.isinf(all_filtered_myX)))
print(np.sum(np.isinf(all_filtered_myY)))

all_myY = all_filtered_myY
all_myX = all_filtered_myX

#all_myX = np.log(1 + all_myX)
#all_myY = np.log(1 + all_myY)
all_myX = np.sqrt(all_myX)
all_myY = np.sqrt(all_myY)

all_myX = sm.add_constant(np.hstack((all_myX, ))) # 

all_diff_model = sm.OLS(all_myY, all_myX).fit()
all_predY = all_diff_model.predict(all_myX)

print(all_diff_model.summary())

plt.scatter(all_myY, all_predY)
#plt.scatter(all_myX, all_myY)
#plt.plot(all_myX, all_predY, 'r--')
plt.show()

In [ ]:
'''
final_predY = all_predY#np.exp(all_predY) - 1
final_myY = all_myY#np.exp(all_myY) - 1

error = np.abs(final_predY - final_myY)

for i in range(0, 100, 5):
    p1 = np.percentile(error, i) # return 50th percentile, e.g median.
    p2 = np.percentile(final_predY, i) # return 50th percentile, e.g median.
    p3 = np.percentile(final_myY, i) # return 50th percentile, e.g median.
    print('%i: %0.3f, %0.2f, %0.2f' % (i, p1, p2, p3))
    
plt.scatter(all_myY, all_predY - all_myY)
plt.show()

'''

In [ ]:
#final_predY = all_predY
#final_myY = all_myY
final_predY = all_predY ** 2
final_myY = all_myY ** 2

#final_predY = np.exp(all_predY) - 1
#final_myY = np.exp(all_myY) - 1

error = np.abs(final_predY - final_myY)

for i in range(0, 100, 2):
    p1 = np.percentile(error, i) # return 50th percentile, e.g median.
    p2 = np.percentile(2 * error / (final_myY + final_predY), i) # return 50th percentile, e.g median.
    p3 = np.percentile(final_myY, i) # return 50th percentile, e.g median.
    p4 = np.percentile(final_predY, i) # return 50th percentile, e.g median.
    print('%i: %0.3f, %0.3f, %0.2f, %0.2f' % (i, p1, p2, p3, p4))
    
plt.title('Predicted vs actual fixed sea lice')
plt.xlabel('Predicted sea lice')
plt.ylabel('Actual sea lice')
plt.scatter(final_predY, final_myY)
plt.show()